
# Watershed Dynamics Model 

<img src="https://www.washington.edu/brand/files/2014/09/W-Logo_Purple_Hex.png" style="float:right;width:300px;padding:20px">   

Here we explore recharge-driven shallow landsliding in the North Cascades National Park Complex (NOCA) from a HydroShare Observatory. 

This Jupyter Notebook runs the Landlab LandslideProbability component on a 30-m digital elevation model (DEM) for NOCA using recharge from Variable Infiltration Capacity (VIC) hydrologic model as described in the paper: <br />
#### Strauch et al. 2017. A hydro-climatological approach to predicting regional landslide probability using Landlab. Earth Surface Dynamics, (in prep). <br />

This notebook performs the following functions to replicate the paper findings:<br >
1) Import libraries and set HydroShare variables<br />
2) Review data needed as input for the landslide model<br />
3) Create a RasterModelGrid based on a 30-m DEM<br />
4) Access and assign data fields used to calculate landslide probability<br />
5) Set Number of iterations to run Monte Carlo simulation<br />
6) Specify recharge option as _data driven spatial_ and access Python dictionaries to generate recharge distributions<br /> 
7) Run Landlab LandslideProbability component<br /> 
8) Display and visualize results of stability analysis<br /> 
9) Save Notebook and Results back to HydroShare<br /> 
<br /> 


## To run this notebook:

To run this example, click in each shaded cell below and "shift + enter" to run each cell. Alternatively, you can run groups of cells by clicking "Cell" on the menu above and selecting you run option. This is also where you can clear outputs from previous runs.

If an error occurs, try "Restart" the kernel by clicking "Kernel" on the menu above.

## 1.  HydroShare Setup and Preparation

To run this notebook, we must import several libraries.
The hs_utils library provides functions for interacting with HydroShare, including resource querying, dowloading. and creation. Additional libraries support the functions of Landlab.

### Import libraries

In [ ]:
#import Python utilities for calculating and plotting
import six
import os
import matplotlib as mpl
mpl.use('agg')
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import cPickle as pickle

#import utilities for importing and exporting to HydroShare
from utilities import hydroshare
# set variables for interacting with HydroShare from this notebook
hs=hydroshare.hydroshare()
# Create object to map the home directory
homedir = r'/home/jovyan/work/notebooks/data/' + str(os.environ["HS_RES_ID"]) + '/' + str(os.environ["HS_RES_ID"]) + '/data/contents/'
print homedir

# Import Landlab libraries
import landslide_probability
from landslide_probability import LandslideProbability
from landlab import imshow_grid_at_node
# from landlab.plot.imshow import imshow_node_grid
from landlab.io import read_esri_ascii
from landlab.io import write_esri_ascii
from collections import defaultdict

If you are curious about where the data is being downloaded, click on the Jupyter Notebook dashboard icon in upper rigth corner to see a File System view.  The homedir directory location printed above is where you can find the data and contents you will download to a HydroShare JupyterHub server.  At the end of this work session, you can migrate this data to the HydroShare iRods server as a Generic Resource. 

### Import data
 This data was preprocessed for the North Cascades National Park Complex case study and is on HydroShare as [Regional landslide hazard using Landlab - NOCA Data](https://www.hydroshare.org/resource/a5b52c0e1493401a815f4e77b09d352b/). Click on the link to see the data repository on HydroShare.

Set a variable name for the data resource using the HydroShare resource ID.

In [ ]:
Data_ResourceID='a5b52c0e1493401a815f4e77b09d352b'

Execute the next cell to download data from another HydroShare resource - this may take a minute.

In [ ]:
hs.getResourceFromHydroShare(Data_ResourceID)
data_folder = '/home/jovyan/work/notebooks/data/'+ Data_ResourceID +'/'+Data_ResourceID+'/data/contents/Data files/'
print('This is the location on the HydroShare JupyterHub server where the data has just been downloaded:')
print data_folder

 ## 2. Review data needed as input for the landslide model


Check the list of data inputs that the component needs.

In [ ]:
sorted(LandslideProbability.input_var_names)

Review the details of what each variable represents.

In [ ]:
LandslideProbability._var_doc

Check the units of each variable.

In [ ]:
LandslideProbability._var_units

Now we will establish a RasterModelGrid based on a DEM for assigning our variables to.
Nodes are the center point of grid cells or pixels that are 30 m by 30 m in this example. 

## 3. Create a RasterModelGrid based on a 30-m DEM

#### Load  landslide model inputs from ASCII textfile (ArcGIS raster conversion) into Landlab grid

Load DEM elevation grid<br />
Set_nodata_nodes_to_inactive that have no data (e.g., -9999), which establishes boundary conditions<br />
This might take a few minutes as the park is large (2,757 km2).

In [ ]:
(grid, z) = read_esri_ascii(data_folder+'/elevation.txt',name='topographic__elevation')
grid.at_node.keys()     # loads DEM grid with elevation
grid.set_nodata_nodes_to_closed(grid.at_node['topographic__elevation'], -9999) # set boundary conditions closed where no data

Confirm the size of the grid, nodes located every 30 m.

In [ ]:
grid.number_of_nodes

Now we will attach data to this grid

## 4. Access and assign data fields to grid used to calculate landslide probability and set boundary conditions

#### For each input below
1. Load data from ascii text file
2. Assign grid to Landlab node
3. Set boundary conditions

For the eSurf paper with NOCA extent, this takes ~60 sec to load each file on NCSA ROGER super computer. 

Load slope


In [ ]:
(grid1, slope) = read_esri_ascii(data_folder+'/slope_tang17d.txt')
grid.add_field('node', 'topographic__slope', slope)
grid.set_nodata_nodes_to_closed(grid.at_node['topographic__slope'], -9999)
grid.set_nodata_nodes_to_closed(grid.at_node['topographic__slope'], 0.0)

Load contributing area

In [ ]:
(grid1, ca) = read_esri_ascii(data_folder+'/cont_area.txt')
grid.add_field('node', 'topographic__specific_contributing_area', ca)
grid.set_nodata_nodes_to_closed(grid.at_node['topographic__specific_contributing_area'], -9999)

Load transmissivity

In [ ]:
(grid1, T) = read_esri_ascii(data_folder+'/transmis.txt')
grid.add_field('node', 'soil__transmissivity', T)
grid.set_nodata_nodes_to_closed(grid.at_node['soil__transmissivity'], -9999)

Load cohesion (mode, min, and max) - this takes ~3 minutes because 3 cohesion fields are provided to create more flexibility in how cohesion is distributed on the landscape.

In [ ]:
(grid1, C) = read_esri_ascii(data_folder+'/cohesion_mode.txt')
C[C == 0.0] = 1.0  # ensure minimum is >0 Pa for use in distributions generation
grid.add_field('node', 'soil__mode_total_cohesion', C)
grid.set_nodata_nodes_to_closed(grid.at_node['soil__mode_total_cohesion'], -9999)

(grid1, C_min) = read_esri_ascii(data_folder+'/cohesion_min.txt')
grid.add_field('node', 'soil__minimum_total_cohesion', C_min)
grid.set_nodata_nodes_to_closed(grid.at_node['soil__minimum_total_cohesion'], -9999)

(grid1, C_max) = read_esri_ascii(data_folder+'/cohesion_max.txt')
grid.add_field('node', 'soil__maximum_total_cohesion', C_max)
grid.set_nodata_nodes_to_closed(grid.at_node['soil__maximum_total_cohesion'], -9999)

Load internal angle of friction

In [ ]:
(grid1, phi) = read_esri_ascii(data_folder+'/frict_angle.txt')
grid.add_field('node', 'soil__internal_friction_angle', phi)
grid.set_nodata_nodes_to_closed(grid.at_node['soil__internal_friction_angle'], -9999)

Set soil density value and assign to all nodes as a constant

In [ ]:
grid['node']['soil__density'] = 2000*np.ones(grid.number_of_nodes)

Load soil thickness or depth

In [ ]:
(grid1, hs) = read_esri_ascii(data_folder+'/soil_depth.txt')
grid.add_field('node', 'soil__thickness', hs)
grid.set_nodata_nodes_to_closed(grid.at_node['soil__thickness'], -9999)

Load analysis mask and actual landslide

In [ ]:
(grid1, mask) = read_esri_ascii(data_folder+'/exclud_mask.txt')
grid.add_field('node', 'exclusion_mask', mask)
grid.set_nodata_nodes_to_closed(grid.at_node['exclusion_mask'], -9999)

Load observed landslide inventory. Class 1-5 are landslides, 8 is no landslide mapped for later plotting

In [ ]:
(grid1, slides) = read_esri_ascii(data_folder+'/landslide_type.txt')
grid.add_field('node', 'landslides', slides)

## 6. Specify recharge option as _data driven spatial_ and access Python dictionaries to generate recharge distributions 

Recharge is this model represents the annual maximum recharge in mm/day. This corresponds to the wettest conditions expected annually, which is the severest soil-saturated conditions likely to occur at least once a year. 

Specify distribution to use in simulation

In [ ]:
distribution = 'data_driven_spatial'

Load pre-processed routed flows dictionaries containing HSD_id and fractional drainage at each node and recharge dictionaries.  HSD is the Hydrologic Source Domain, which is the VIC data in this case study at ~5x6 km2 grid size.  The 'pickle' utility loads existing dictionaries.

In [ ]:
# dict of node id (key) and HSD_ids (values)
HSD_id_dict = pickle.load(open(data_folder+'/dict_uniq_ids.p', 'rb'))
# dict of node id (key) and fractions (values)
fract_dict = pickle.load(open(data_folder+'/dict_coeff.p', 'rb'))
# dict of HSD id (key) with arrays of recharge (values)
HSD_dict = pickle.load(open(data_folder+'/HSD_dict.p', 'rb'))


Combine dictionaries into __ordered__ parameters required for _data driven spatial_ distribution

In [ ]:
HSD_inputs = [HSD_dict,HSD_id_dict, fract_dict]

## 5. Set Number of iterations to run Monte Carlo simulation

The landslide component employes the infinite slope model to calculate factor-of-safety index values using a Monte Carlo simulation, which randomly selects input values from parameter distributions. You can specify the number of iterations to run Monte Carlo simulation, but the default is 250. The higher the number of iteration, the longer the program runs, but the more precise the probability of failure results become.

In [ ]:
iterations = 3000

## 7. Run the Landlab LandslideProbability Component

To run the landslide model, we first instantiate the LandslideProbability component with the above parameters, as well as the grid and number of iterations we specified before. Instantiate creates an instance of a class. (For example, the iPhone is a class and each phone is an instance.)

No outputs are generated by this command as it is setting up the recharge and instantiating the component.

In [ ]:
LS_prob = LandslideProbability(grid,
    number_of_iterations=iterations,
    groudwater__recharge_distribution=distribution,
    groudwater__recharge_HSD_inputs=HSD_inputs)

Once the component has been instantiated, we generate outputs from running the component by calling the component's 'calculate_landslide_probability' method using the class instance (e.g., LS_prob). The following cell runs the model; in the following section we will assessing the results. These calculations will take a few minutes given the size of the modeling domain represented by core nodes.

In [ ]:
LS_prob.calculate_landslide_probability()
print('Landslide probability successfully calculated')

The outputs of landslide model simulation are:

In [ ]:
sorted(LS_prob.output_var_names)

Save the arrays as variables by 'attaching the fields to the grid' and view the outputs. **component already does this**

This simulation generates a probability value for each core node. 

In [ ]:
LS_prob_probability_of_failure = grid.at_node['landslide__probability_of_failure']
grid.at_node['landslide__probability_of_failure']

This simulation generates a mean relative wetness value for each core node as well.

In [ ]:
LS_prob_relative_wetness = grid.at_node['soil__mean_relative_wetness']
grid.at_node['soil__mean_relative_wetness']

## 8. Display and visualize results of stability analysis

Set plotting parameters

In [ ]:
mpl.rcParams['xtick.labelsize'] = 15
mpl.rcParams['ytick.labelsize'] = 15
mpl.rcParams['lines.linewidth'] = 1
mpl.rcParams['axes.labelsize'] = 18
mpl.rcParams['legend.fontsize'] = 15

Plot elevation

In [ ]:
plt.figure('Elevations from the DEM [m]')
imshow_grid_at_node(grid, 'topographic__elevation', cmap='terrain',
                 grid_units=('coordinates', 'coordinates'),
                 shrink=0.75, var_name='Elevation', var_units='m')
#plt.savefig('NOCA_elevation.png')

Excluded areas from the analysis are shown in black, including outside the park and inside the park areas that are water bodies, snow, glaciers, wetlands, exposed bedrock, and slopes <= 17 degrees. 

Plot slope overlaid with mapped landslide types. Takes about a few minutes.

In [ ]:
plt.figure('Landslides')
ls_mask1 = grid.at_node['landslides'] != 1.0
ls_mask2 = grid.at_node['landslides'] != 2.0
ls_mask3 = grid.at_node['landslides'] != 3.0
ls_mask4 = grid.at_node['landslides'] != 4.0
overlay_landslide1 = np.ma.array(grid.at_node['landslides'], mask=ls_mask1)
overlay_landslide2 = np.ma.array(grid.at_node['landslides'], mask=ls_mask2)
overlay_landslide3 = np.ma.array(grid.at_node['landslides'], mask=ls_mask3)
overlay_landslide4 = np.ma.array(grid.at_node['landslides'], mask=ls_mask4)
imshow_grid_at_node(grid, 'topographic__slope', cmap='pink',
                 grid_units=('coordinates', 'coordinates'), vmax=2.,
                 shrink=0.75, var_name='Slope', var_units='m/m')
imshow_grid_at_node(grid, overlay_landslide1, color_for_closed='None',
                 allow_colorbar=False, cmap='cool')
imshow_grid_at_node(grid, overlay_landslide2, color_for_closed='None',
                 allow_colorbar=False, cmap='autumn')
imshow_grid_at_node(grid, overlay_landslide3, color_for_closed='None',
                 allow_colorbar=False, cmap='winter')
imshow_grid_at_node(grid, overlay_landslide4, color_for_closed='None',
                 allow_colorbar=False,cmap='summer')
#plt.savefig('NOCA_Landslides_on_Slope.png')

Legend to mapped landslides: blue - debris avalanches, cyan - falls/topples, red - debris torrents, and green - slumps/creeps

Plot of soil depth (m)

In [ ]:
plt.figure('Soil Thickness')
imshow_grid_at_node(grid, 'soil__thickness', cmap='copper_r',
                 grid_units=('coordinates', 'coordinates'), shrink=0.75,
                 var_name='Soil Thickness', var_units='m')
#plt.savefig('NOCA_SoilDepth.png')

Plot probability of saturation 

In [ ]:
plt.figure('Probability of Saturation')
imshow_grid_at_node(grid, 'soil__probability_of_saturation', cmap='YlGnBu',
                 limits=((0), (1)),
                 grid_units=('coordinates', 'coordinates'),
                 shrink=0.75, var_name='Probability of Saturation',
                 var_units='no units')
#plt.savefig('NOCA_ProbabilityofSaturation.png')

This map shows the probability of saturation as high throughout much of the area because we modeled the annual maximum recharge, which is esssentially the worst case conditions that might lead to instability.

Plot probability of failure; Compare this with the elevation and slope maps.

In [ ]:
plt.figure('Probability of Failure')
imshow_grid_at_node(grid, 'landslide__probability_of_failure', cmap='OrRd',
                 grid_units=('coordinates', 'coordinates'), shrink=0.75,
                 var_name='Probability of Failure', var_units='no units')
#plt.savefig('NOCA_ProbabilityofFailure.png')

The map of probability of failure shows higher probabilities at higher elevations below retreating glaciers where vegation is sparse and shallow unconsolidated sediment is prevalent.

To review the fields assigned to the grid, simply execute the following command.

In [ ]:
grid.at_node

Export data from model run: FS probability, mean Reletive wetness, probability of saturation

In [ ]:
import pandas as pd
core_nodes = grid.core_nodes
data_extracted = {'PF_3S_SD': np.array(
                 grid.at_node['landslide__probability_of_failure'][grid.core_nodes]),
                 'mean_RW': np.array(grid.at_node['soil__mean_relative_wetness']
                 [grid.core_nodes]),'prob_sat': np.array(
                 grid.at_node['soil__probability_of_saturation'][grid.core_nodes])}
headers = ['PF_3S_SD','mean_RW','prob_sat']
df = pd.DataFrame(data_extracted, index=core_nodes, columns=(headers))
df.to_csv('FS1k_SSD_ddsp.csv')

Make ascii files for raster creation in GIS

In [ ]:
write_esri_ascii('prbF_1kSSD_ddsp.txt',grid,names='landslide__probability_of_failure')
write_esri_ascii('mRW_1kSSD_ddsp.txt',grid,names='soil__mean_relative_wetness')
write_esri_ascii('prbSat_1kSSD_ddsp.txt',grid,names='soil__probability_of_saturation')

## 9. Save the results back into HydroShare
<a name="creation"></a>

Using the `hs_utils` library, the results of the Geoprocessing steps above can be saved back into HydroShare.  First, define all of the required metadata for resource creation, i.e. *title*, *abstract*, *keywords*, *content files*.  In addition, we must define the type of resource that will be created, in this case *genericresource*.  

***Note:*** Make sure you save the notebook at this point, so that all notebook changes will be saved into the new HydroShare resource.


***Option A*** : define the resource from which this "NEW" content has been derived.  This is one method for tracking resource provenance.

Create list of files to save to HydroShare. Verify location and names.

In [ ]:
ThisNotebook='NOCA_runGMDpaper_LandlabLandslide.ipynb' #check name for consistency
files=[homedir+ ThisNotebook,
       homedir+'FS1k_SSD_ddsp.csv',
       homedir+ 'prbF_1kSSD_ddsp.txt',
       homedir+ 'mRW_1kSSD_ddsp.txt',
       homedir+ 'prbSat_1kSSD_ddsp.txt']
print files #print location and names of files to save

In [ ]:
# for each file downloaded onto the server folder, move to a new HydroShare Generic Resource
title = 'Landslide Model run SSURGO-SD n=1000 for eSurf paper from NOCA Observatory' # title for the new resource
abstract = 'This a reproducible demonstration of the landslide modeling results from eSurf paper: Strauch et al. (2017) ' # abstract for the new resource
keywords = ['landslide', 'climate', 'VIC','saturation','relative wetness'] # keywords for the new resource
rtype = 'genericresource'          # Hydroshare resource type

# create the new resource
resource_id = hs.createHydroShareResource(abstract, 
                                          title,
                                          keywords=keywords, 
                                          resource_type=rtype, 
                                          content_files=files, 
                                          public=False)